# The Inflationary Backdrop
## Lighthouse Macro | Macro, Illuminated.

This notebook recreates the inflation dashboard using Python and FRED data.

In [ ]:
# Install required packages if not already installed
# !pip install pandas matplotlib plotly fredapi numpy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
from fredapi import Fred
import warnings
warnings.filterwarnings('ignore')

# Set up FRED API
FRED_API_KEY = '6dcc7a0d790cdcc28c1f751420ee9d27'
fred = Fred(api_key=FRED_API_KEY)

# Color scheme
OCEAN_BLUE = '#0066CC'
DEEP_ORANGE = '#FF6B35'
NEON_CAROLINA_BLUE = '#00C5FF'
NEON_MAGENTA = '#FF00FF'
MED_LIGHT_GRAY = '#999999'

In [ ]:
def create_chart_template():
    """Create a standard template for all charts"""
    return {
        'layout': {
            'font': {'family': 'system-ui, -apple-system, sans-serif', 'size': 11, 'color': MED_LIGHT_GRAY},
            'plot_bgcolor': 'white',
            'paper_bgcolor': 'white',
            'showlegend': True,
            'legend': {'x': 0, 'y': 1.1, 'orientation': 'h'},
            'margin': {'l': 50, 'r': 50, 't': 80, 'b': 50},
            'annotations': [{
                'text': 'MACRO, ILLUMINATED.',
                'xref': 'paper', 'yref': 'paper',
                'x': 0.98, 'y': 0.02,
                'xanchor': 'right', 'yanchor': 'bottom',
                'font': {'size': 9, 'color': OCEAN_BLUE},
                'showarrow': False
            }]
        }
    }

def calculate_yoy_change(series, periods=12):
    """Calculate year-over-year percentage change"""
    return ((series / series.shift(periods) - 1) * 100).round(2)

print("Setup complete! Ready to fetch data and create charts.")

## Chart 1: Inflation Re-emerges as the Constraint

In [ ]:
# Fetch CPI data
cpi = fred.get_series('CPIAUCSL', start='1990-01-01')
core_cpi = fred.get_series('CPILFESL', start='1990-01-01')

# Calculate year-over-year changes
cpi_yoy = calculate_yoy_change(cpi)
core_cpi_yoy = calculate_yoy_change(core_cpi)

# Create chart
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=cpi_yoy.index, y=cpi_yoy,
    mode='lines', name='CPI (All Items)',
    line=dict(color=OCEAN_BLUE, width=2.5)
))

fig.add_trace(go.Scatter(
    x=core_cpi_yoy.index, y=core_cpi_yoy,
    mode='lines', name='Core CPI',
    line=dict(color=DEEP_ORANGE, width=2.5)
))

# Add 2% reference line
fig.add_hline(y=2, line_dash="dash", line_color=MED_LIGHT_GRAY)

template = create_chart_template()
fig.update_layout(
    title='Chart 1: Inflation Re-emerges as the Constraint<br><sub>U.S. CPI inflation surged above 9% in 2022—the highest since the early 1980s</sub>',
    yaxis_title='Year-over-Year %',
    height=500,
    **template['layout']
)

fig.show()

## Chart 2: Demand Overshoot—Disposable Personal Income

In [ ]:
# Fetch disposable personal income
dpi = fred.get_series('DSPIC96', start='1990-01-01')

# Index to 1990 = 100
dpi_indexed = (dpi / dpi.iloc[0] * 100).round(1)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=dpi_indexed.index, y=dpi_indexed,
    mode='lines', name='Real Disposable Income',
    line=dict(color=OCEAN_BLUE, width=2.5),
    fill='tonexty', fillcolor=f'rgba(0, 102, 204, 0.2)'
))

template = create_chart_template()
fig.update_layout(
    title='Chart 2: Demand Overshoot—Disposable Personal Income<br><sub>Fiscal transfers during COVID pushed disposable income far above trend</sub>',
    yaxis_title='Index (1990 = 100)',
    height=500,
    **template['layout']
)

fig.show()

## Chart 3: Supply Under Strain—Global Supply Chain Pressure

In [ ]:
# Fetch Global Supply Chain Pressure Index
gscpi = fred.get_series('GSCPI', start='2000-01-01')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=gscpi.index, y=gscpi,
    mode='lines', name='Global Supply Chain Pressure Index',
    line=dict(color=DEEP_ORANGE, width=2.5)
))

template = create_chart_template()
fig.update_layout(
    title='Chart 3: Supply Under Strain—Global Supply Chain Pressure<br><sub>Global supply chain disruptions spiked during the pandemic</sub>',
    yaxis_title='Index (Std Dev)',
    height=500,
    **template['layout']
)

fig.show()

## Chart 4: Energy and Commodity Shock

In [ ]:
# Fetch energy data
wti = fred.get_series('DCOILWTICO', start='2000-01-01')
natgas = fred.get_series('DHHNGSP', start='2000-01-01')

# Create subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=wti.index, y=wti, mode='lines', name='WTI Crude Oil',
               line=dict(color=OCEAN_BLUE, width=2.5)),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=natgas.index, y=natgas, mode='lines', name='Natural Gas',
               line=dict(color=DEEP_ORANGE, width=2.5)),
    secondary_y=True
)

fig.update_yaxes(title_text="WTI Crude ($/barrel)", secondary_y=False)
fig.update_yaxes(title_text="Natural Gas ($/MMBtu)", secondary_y=True)

template = create_chart_template()
fig.update_layout(
    title='Chart 4: Energy and Commodity Shock<br><sub>Russia\'s invasion of Ukraine in 2022 created a massive energy shock</sub>',
    height=500,
    **template['layout']
)

fig.show()

## Chart 5: Supply vs Demand—GSCPI and CPI

In [ ]:
# Get CPI for the same period as GSCPI
cpi_2000 = fred.get_series('CPIAUCSL', start='2000-01-01')
cpi_2000_yoy = calculate_yoy_change(cpi_2000)

# Align dates
common_dates = gscpi.index.intersection(cpi_2000_yoy.index)
gscpi_aligned = gscpi.loc[common_dates]
cpi_aligned = cpi_2000_yoy.loc[common_dates]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=gscpi_aligned.index, y=gscpi_aligned, mode='lines', 
               name='Supply Chain Pressure', line=dict(color=DEEP_ORANGE, width=2.5)),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=cpi_aligned.index, y=cpi_aligned, mode='lines', 
               name='CPI Inflation', line=dict(color=OCEAN_BLUE, width=2.5)),
    secondary_y=True
)

fig.update_yaxes(title_text="GSCPI (Std Dev)", secondary_y=False)
fig.update_yaxes(title_text="CPI YoY %", secondary_y=True)

template = create_chart_template()
fig.update_layout(
    title='Chart 5: Supply vs Demand—GSCPI and CPI<br><sub>Supply normalized but inflation remained elevated—demand and services driving persistence</sub>',
    height=500,
    **template['layout']
)

fig.show()

## Chart 6: Goods vs Services Inflation—The Last Mile Problem

In [ ]:
# Fetch goods and services CPI
goods_cpi = fred.get_series('CUSR0000SAC', start='2000-01-01')
services_cpi = fred.get_series('CUSR0000SAS', start='2000-01-01')

# Calculate YoY changes
goods_yoy = calculate_yoy_change(goods_cpi)
services_yoy = calculate_yoy_change(services_cpi)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=goods_yoy.index, y=goods_yoy,
    mode='lines', name='Goods Inflation',
    line=dict(color=NEON_CAROLINA_BLUE, width=2.5)
))

fig.add_trace(go.Scatter(
    x=services_yoy.index, y=services_yoy,
    mode='lines', name='Services Inflation',
    line=dict(color=NEON_MAGENTA, width=2.5)
))

fig.add_hline(y=2, line_dash="dash", line_color=MED_LIGHT_GRAY)

template = create_chart_template()
fig.update_layout(
    title='Chart 6: Goods vs Services Inflation—The Last Mile Problem<br><sub>Goods inflation reversed as supply chains healed, services inflation remained sticky</sub>',
    yaxis_title='Year-over-Year %',
    height=500,
    **template['layout']
)

fig.show()

## Chart 7: Inflation Expectations—Survey vs Market

In [ ]:
# Fetch inflation expectations
michigan = fred.get_series('MICH', start='2000-01-01')
breakeven_5y = fred.get_series('T5YIE', start='2000-01-01')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=michigan.index, y=michigan,
    mode='lines', name='Michigan Survey',
    line=dict(color=DEEP_ORANGE, width=2.5)
))

fig.add_trace(go.Scatter(
    x=breakeven_5y.index, y=breakeven_5y,
    mode='lines', name='5Y Breakeven',
    line=dict(color=OCEAN_BLUE, width=2.5)
))

fig.add_hline(y=2, line_dash="dash", line_color=MED_LIGHT_GRAY)

template = create_chart_template()
fig.update_layout(
    title='Chart 7: Inflation Expectations—Survey vs Market<br><sub>Survey-based expectations rose faster than market-based measures</sub>',
    yaxis_title='Expected Inflation (%)',
    height=500,
    **template['layout']
)

fig.show()

## Chart 8: Goods Reversal—Durable Goods Deflation

In [ ]:
# Fetch durable goods CPI
durables_cpi = fred.get_series('CUSR0000SAD', start='2000-01-01')
durables_yoy = calculate_yoy_change(durables_cpi)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=durables_yoy.index, y=durables_yoy,
    mode='lines', name='Durable Goods CPI',
    line=dict(color=NEON_CAROLINA_BLUE, width=2.5),
    fill='tonexty', fillcolor=f'rgba(0, 197, 255, 0.2)'
))

fig.add_hline(y=0, line_dash="dash", line_color=MED_LIGHT_GRAY)

template = create_chart_template()
fig.update_layout(
    title='Chart 8: Goods Reversal—Durable Goods Deflation<br><sub>Durable goods prices surged during pandemic, then reversed sharply</sub>',
    yaxis_title='Year-over-Year %',
    height=500,
    **template['layout']
)

fig.show()

## Chart 9: Shelter Stickiness—CPI vs Market Rents

In [ ]:
# Fetch shelter CPI and Zillow rent index
shelter_cpi = fred.get_series('CUSR0000SAH', start='2000-01-01')
zillow_rent = fred.get_series('ZRIMUS', start='2015-01-01')

# Calculate YoY changes
shelter_yoy = calculate_yoy_change(shelter_cpi)
zillow_yoy = calculate_yoy_change(zillow_rent)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=shelter_yoy.index, y=shelter_yoy,
    mode='lines', name='CPI Shelter',
    line=dict(color=OCEAN_BLUE, width=2.5)
))

fig.add_trace(go.Scatter(
    x=zillow_yoy.index, y=zillow_yoy,
    mode='lines', name='Zillow Rent Index',
    line=dict(color=DEEP_ORANGE, width=2.5)
))

fig.add_hline(y=2, line_dash="dash", line_color=MED_LIGHT_GRAY)

template = create_chart_template()
fig.update_layout(
    title='Chart 9: Shelter Stickiness—CPI vs Market Rents<br><sub>Private rental data shows rents peaked, but official CPI shelter remains elevated due to lags</sub>',
    yaxis_title='Year-over-Year %',
    height=500,
    **template['layout']
)

fig.show()

## Chart 10: Wage Growth—The Services Inflation Channel

In [ ]:
# Fetch average hourly earnings
wages = fred.get_series('CES0500000003', start='1990-01-01')
wages_yoy = calculate_yoy_change(wages)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=wages_yoy.index, y=wages_yoy,
    mode='lines', name='Average Hourly Earnings',
    line=dict(color=OCEAN_BLUE, width=2.5),
    fill='tonexty', fillcolor=f'rgba(0, 102, 204, 0.2)'
))

fig.add_hline(y=2, line_dash="dash", line_color=MED_LIGHT_GRAY)

template = create_chart_template()
fig.update_layout(
    title='Chart 10: Wage Growth—The Services Inflation Channel<br><sub>Wage growth has slowed but remains elevated compared to the 2010s</sub>',
    yaxis_title='Year-over-Year %',
    height=500,
    **template['layout']
)

fig.show()

## Chart 11: Policy Response—Fed Funds Rate Hiking Cycle

In [ ]:
# Fetch Fed Funds Rate and Real Fed Funds Rate
fed_funds = fred.get_series('FEDFUNDS', start='1980-01-01')
real_fed_funds = fred.get_series('REAINTRATREARAT1YE', start='1980-01-01')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=fed_funds.index, y=fed_funds,
    mode='lines', name='Nominal Fed Funds',
    line=dict(color=OCEAN_BLUE, width=2.5)
))

fig.add_trace(go.Scatter(
    x=real_fed_funds.index, y=real_fed_funds,
    mode='lines', name='Real Fed Funds',
    line=dict(color=DEEP_ORANGE, width=2.5)
))

fig.add_hline(y=0, line_dash="dash", line_color=MED_LIGHT_GRAY)

template = create_chart_template()
fig.update_layout(
    title='Chart 11: Policy Response—Fed Funds Rate Hiking Cycle<br><sub>The Fed hiked faster than any time since the Volcker era</sub>',
    yaxis_title='Interest Rate (%)',
    height=500,
    **template['layout']
)

fig.show()

## Chart 12: Fiscal Math—Debt and Interest Expense

In [ ]:
# Fetch debt and GDP data
federal_debt = fred.get_series('GFDEBTN', start='1990-01-01')
gdp = fred.get_series('GDP', start='1990-01-01')
interest_exp = fred.get_series('A091RC1Q027SBEA', start='1990-01-01')

# Calculate debt to GDP ratio
debt_to_gdp = (federal_debt / gdp * 100).round(1)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=debt_to_gdp.index, y=debt_to_gdp, mode='lines', 
               name='Debt to GDP', line=dict(color=OCEAN_BLUE, width=2.5),
               fill='tonexty', fillcolor=f'rgba(0, 102, 204, 0.2)'),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=interest_exp.index, y=interest_exp, mode='lines', 
               name='Interest Expense', line=dict(color=DEEP_ORANGE, width=2.5)),
    secondary_y=True
)

fig.update_yaxes(title_text="Debt/GDP (%)", secondary_y=False)
fig.update_yaxes(title_text="Interest Exp ($ Bil)", secondary_y=True)

template = create_chart_template()
fig.update_layout(
    title='Chart 12: Fiscal Math—Debt and Interest Expense<br><sub>Federal debt remains elevated, interest expense climbing with higher rates</sub>',
    height=500,
    **template['layout']
)

fig.show()

## Chart 13: Deglobalization—Shifting Trade Flows

In [ ]:
# Fetch trade data (using available proxy)
try:
    china_imports = fred.get_series('IR4Q', start='2000-01-01')
    total_imports = fred.get_series('IMPGS', start='2000-01-01')
    
    # Calculate China's share (this is a simplified calculation)
    # Note: The actual calculation would require more specific trade data
    china_share = (china_imports / total_imports * 100).round(1)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=china_share.index, y=china_share,
        mode='lines', name='China Share of US Imports',
        line=dict(color=DEEP_ORANGE, width=2.5),
        fill='tonexty', fillcolor=f'rgba(255, 107, 53, 0.2)'
    ))
    
    template = create_chart_template()
    fig.update_layout(
        title='Chart 13: Deglobalization—Shifting Trade Flows<br><sub>China\'s share of U.S. imports declining as firms de-risk supply chains</sub>',
        yaxis_title='Share of Total Imports (%)',
        height=500,
        **template['layout']
    )
    
    fig.show()
except:
    print("Trade flow data not available with this API key configuration")

## Chart 14: Energy Transition—Commodity Intensity

In [ ]:
# Fetch commodity prices
copper = fred.get_series('PCOPPUSDM', start='2000-01-01')
palladium = fred.get_series('PPALLUSDM', start='2000-01-01')

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=copper.index, y=copper, mode='lines', 
               name='Copper Price', line=dict(color=DEEP_ORANGE, width=2.5)),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=palladium.index, y=palladium, mode='lines', 
               name='Palladium Price', line=dict(color=NEON_CAROLINA_BLUE, width=2.5)),
    secondary_y=True
)

fig.update_yaxes(title_text="Copper ($/MT)", secondary_y=False)
fig.update_yaxes(title_text="Palladium ($/oz)", secondary_y=True)

template = create_chart_template()
fig.update_layout(
    title='Chart 14: Energy Transition—Commodity Intensity<br><sub>Green transition requires massive investment in metals like copper and palladium</sub>',
    height=500,
    **template['layout']
)

fig.show()

## Chart 15: Demographics—Labor Force Participation

In [ ]:
# Fetch labor force participation data
total_lfpr = fred.get_series('CIVPART', start='1990-01-01')
prime_age_lfpr = fred.get_series('LNS11300060', start='1990-01-01')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=total_lfpr.index, y=total_lfpr,
    mode='lines', name='Total LFPR',
    line=dict(color=OCEAN_BLUE, width=2.5)
))

fig.add_trace(go.Scatter(
    x=prime_age_lfpr.index, y=prime_age_lfpr,
    mode='lines', name='Prime Age (25-54)',
    line=dict(color=NEON_MAGENTA, width=2.5)
))

template = create_chart_template()
fig.update_layout(
    title='Chart 15: Demographics—Labor Force Participation<br><sub>Prime-age participation rebounded, but aging demographics dominate long run</sub>',
    yaxis_title='Participation Rate (%)',
    yaxis=dict(range=[58, 85]),
    height=500,
    **template['layout']
)

fig.show()

## Summary

This dashboard illustrates the complex inflationary backdrop facing policymakers:

- **Demand factors**: Fiscal stimulus created demand overshoot
- **Supply factors**: Global supply chains faced unprecedented strain
- **Energy shocks**: Geopolitical events amplified commodity price volatility
- **Structural shifts**: Deglobalization and energy transition create new dynamics
- **Policy response**: Aggressive tightening aims to restore price stability

The "last mile" of disinflation remains challenging as services inflation proves sticky, driven by wage growth and shelter costs.

---

**Data Sources:** Federal Reserve Economic Data (FRED), Federal Reserve Bank of St. Louis, Bureau of Labor Statistics, Federal Reserve Bank of New York, Zillow Research.

**Disclaimer:** This report is for informational and educational purposes only. It is not financial advice.